Uptrail Week 1 project

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:/Users/HP/Desktop/Uptrail/Week1 Dataset/customer_signups.csv")
df.head(5)

,customer_id,name,email,signup_date,source,region,plan_selected,marketing_opt_in,age,gender
0,CUST00000,Joshua Bryant,NaN,NaN,Instagram,NaN,basic,No,34,Female
1,CUST00001,Nicole Stewart,nicole1@example.com,02-01-2024,LinkedIn,West,basic,Yes,29,Male
2,CUST00002,Rachel Allen,rachel2@example.com,03-01-2024,Google,North,PREMIUM,Yes,34,Non-Binary
3,CUST00003,Zachary Sanchez,zachary3@mailhub.org,04-01-2024,YouTube,NaN,Pro,No,40,Male
4,CUST00004,NaN,matthew4@mailhub.org,05-01-2024,LinkedIn,West,Premium,No,25,Other


In [ ]:
df.dtypes

customer_id         object
name                object
email               object
signup_date         object
source              object
region              object
plan_selected       object
marketing_opt_in    object
age                 object
gender              object
dtype: object

In [ ]:
df['signup_date'] = pd.to_datetime(df['signup_date'])
df.dtypes


customer_id                 object
name                        object
email                       object
signup_date         datetime64[ns]
source                      object
region                      object
plan_selected               object
marketing_opt_in            object
age                         object
gender                      object
dtype: object

In [3]:
bad_values = ['not a date', '', 'n/a']
df['signup_date'] = df['signup_date'].replace(bad_values, np.nan)
df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce')
df['signup_date'].head(20)

0           NaT
1    2024-02-01
2    2024-03-01
3    2024-04-01
4    2024-05-01
5    2024-06-01
6    2024-07-01
7    2024-08-01
8    2024-09-01
9    2024-10-01
10   2024-11-01
11   2024-12-01
12          NaT
13          NaT
14          NaT
15          NaT
16          NaT
17          NaT
18          NaT
19          NaT
Name: signup_date, dtype: datetime64[ns]

In [6]:
#converting marketing-opt_in column into boolean data type and handling missing values
#converting marketing-opt_in column into boolean data type and handling missing values
df['marketing_opt_in'] = df['marketing_opt_in'].astype(str).str.lower().str.strip()
df['marketing_opt_in'].head(5)
df['marketing_opt_in'] = df['marketing_opt_in'].map({
    'yes': True,
    'no' : False
})
df.dtypes

customer_id                 object
name                        object
email                       object
signup_date         datetime64[ns]
source                      object
region                      object
plan_selected               object
marketing_opt_in            object
age                         object
gender                      object
dtype: object

In [9]:
#converting marketing-opt_in column into boolean data type and handling missing values
df['marketing_opt_in'] = df['marketing_opt_in'].astype(str).str.lower().str.strip()
df['marketing_opt_in'].head(5)
df['marketing_opt_in'] = df['marketing_opt_in'].map({
    'yes': True,
    'no' : False
})